## Finding example phrases and forms for annotation

Task: Annotate 1000 random obliques that appear in a locative case in the corpus for concrete locations, abstract locations and time. For better annotation examples of how the words appeared in the corpus are needed. This notebook is for searching for usage examples for words from [a database file created on the basis of the Estonian Reference corpus](https://github.com/estnltk/syntax_experiments/tree/verb_templates/verb_transactions/v33).

The possible location words are from kohasonad_test.csv, which was made with v01_locations_by_verb.ipynb. Here we take 1000 random lemmas from that file to annotate. For ease of annotation we add the form and verbphrase the word appeared in

In [22]:
import pandas as pd
import random
import sqlite3

In [57]:
# andmebaasi failinimi, kust andmeid loetake
filename = "C:\\Users\\kertu.saul\\OneDrive - Eesti Keele Instituut\\Dokumendid\\doktoritoo\\ressurssid\\rektsioonid\\katrin\\andmebaasifailid\\v33_koondkorpus_transaktsioonid.db"

# andmebaasiga ühenduse loomine
conn = sqlite3.connect(filename)
cursor = conn.cursor()

In [94]:
#tagastab andmebaasist lemma, sõnavormi ja näitefraasi iga kasutaja määratud sõna kohta, mis kasutaja määratud verbiga esineb 
def kohad_naidetega(verbid):
    query = (f"SELECT lemma, transaction_row.form, phrase FROM `transaction_row` JOIN `transaction_head` ON transaction_head.id = transaction_row.head_id "
             f"WHERE verb IN ({','.join('?' for _ in verbid)}) AND lemma IN (SELECT lemma FROM temp_lemmad) AND transaction_row.deprel = 'obl' "
             f"AND (transaction_row.feats LIKE ? OR transaction_row.feats LIKE ? OR transaction_row.feats LIKE ? OR transaction_row.feats LIKE ? OR transaction_row.feats LIKE ? OR transaction_row.feats LIKE ? OR transaction_row.feats LIKE ?) ")
    cursor.execute(query, verbid + ('%adit%', '%ill%', '%in%', '%el%', '%all%', '%ad%', '%abl%'))
    kohad_naidetega = list(cursor.fetchall())
    return kohad_naidetega

In [79]:
#võtame 20000 kohakäändes sõnast märgendamiseks 1000 random sõna
kohakaandes = pd.read_csv('C:\\Users\\kertu.saul\\OneDrive - Eesti Keele Instituut\\Dokumendid\\doktoritoo\\fyysilised_kohad\\syntax_experiments_semantic_labelling\\physical_location_labelling\\results\\kohasonad_test.csv')
sonad = kohakaandes['Unnamed: 0'].tolist() #words out
sonad_1000 = random.sample(sonad, 1000) #1000 random words
sonad_1000 = tuple(sonad_1000)

In [96]:
#määrame verbid, mille alluvate seast sõnu ja näiteid otsitakse
verbid = ('jooksma', 'kõndima', 'tõttama', 'istuma', 'käima', 'astuma', 'liikuma', 'lahkuma')
#loome 1000 sõnaga ajutise tabeli, et sealt pärast päritavaid sõnu filtreerida
cursor.execute("CREATE TEMPORARY TABLE temp_lemmad (lemma TEXT)")
cursor.executemany("INSERT INTO temp_lemmad (lemma) VALUES (?)", [(lemma,) for lemma in sonad_1000])
#teostame andmebaasipäringu, tulemuseks list tupleitest kus on lemma, vorm ja näitefraas
naited = kohad_naidetega(verbid)
#kustutame ajutise lemmadega tabeli
cursor.execute("DROP TABLE temp_lemmad")

In [99]:
#paneme tulemused dataframei
naite_df = pd.DataFrame(naited, columns =['lemma', 'form', 'naitelause'])
#võtame iga sõna jaoks ainult esimese näite
naited_unique = naite_df.drop_duplicates(subset='lemma', keep='first')
naited_unique

,lemma,form,naitelause
0,mina,Mul,Mul käivad tihti peal tujud
1,kodu,kodus,kes jõulude ajal ajal kodus käis
2,baar,baaris,Istusin baaris
6,pooltund,pooltunnil,Meediamagnaat LUIK lahkus koos koos kaaslase S...
8,mõte,mõttes,käib mõttes tema pärast pärast vahet
...,...,...,...
16310,am,am,tglt am lahkun
16324,maurus,mauruses,mina olen see nädal korda mauruses käinud
16558,ltsimaja,ltsimajas,mehed käivad ltsimajas
16594,ristimine,ristimisel,ma käisin täna ristimisel


In [100]:
#salvestame csv faili 
teekond = 'C:\\Users\\kertu.saul\\OneDrive - Eesti Keele Instituut\\Dokumendid\\doktoritoo\\fyysilised_kohad\\syntax_experiments_semantic_labelling\\physical_location_labelling\\results\\kohasonad_naited.csv'
naited_unique.to_csv(teekond)

## Tagging words as places
I'll be using 3 tags: 
* time: words denoting time such as ...
* place_c: concrete places that physically exist, for example
    * placenames: Bristol, Sepphoris
    * buildings: pangamaja
    * physical objects: esikohapoodium
* place_a: abstract places
    * things that aren't visible but do exist: Wifi
    * activities/events you have to physically go to: tantsutrenn, missivalimised 

tags: time, place_c (concrete place), place_a (abstract place) 